In [1]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from training_utils import LinearSubspace, symmetric_cross_entropy
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
root = "data/baby"

image_feat = np.load(f"{root}/image_feat.npy")
v_prime_feat = np.load(f"{root}/v_prime_feat.npy")
text_feat = np.load(f"{root}/text_feat.npy")
t_prime_feat = np.load(f"{root}/t_prime_feat.npy")
typographic_feat = np.load(f"{root}/typographic_feat.npy")

class_id = np.arange(len(image_feat))
real_or_fake = np.ones(len(image_feat))  # Dummy values for compatibility

# Convert to float32 if necessary
image_feat = image_feat.astype(np.float32)
v_prime_feat = v_prime_feat.astype(np.float32)
text_feat = text_feat.astype(np.float32)
t_prime_feat = t_prime_feat.astype(np.float32)
typographic_feat = typographic_feat.astype(np.float32)

In [3]:
class FeatureDataset(Dataset):
    def __init__(self, img_i, img_t, img_ti, txt_i, txt_t, class_id, real_or_fake):
        self.img_i = torch.tensor(img_i)
        self.img_t = torch.tensor(img_t)
        self.img_ti = torch.tensor(img_ti)
        self.txt_i = torch.tensor(txt_i)
        self.txt_t = torch.tensor(txt_t)
        self.class_id = torch.tensor(class_id)
        self.real_or_fake = torch.tensor(real_or_fake)

    def __len__(self):
        return len(self.img_i)

    def __getitem__(self, idx):
        return (self.img_i[idx], self.img_t[idx], self.img_ti[idx],
                self.txt_i[self.class_id[idx]], self.txt_t[idx],
                self.class_id[idx], self.real_or_fake[idx])

In [37]:
from training_utils import AverageMeter
import torch.nn.functional as F

# 하이퍼파라미터
D_shared = 64  # Bottleneck dim
out_dim = 64     # 최종 투영 차원
batch_size = 128
num_epochs = 100
initial_lr = 1e-4

# 모델 정의
model = LinearSubspace(out_dim=out_dim, input_size=D_shared).to(device)
v_proj = torch.nn.Linear(4096, D_shared).to(device)
t_proj = torch.nn.Linear(384, D_shared).to(device)

# 옵티마이저 및 스케줄러
optimizer = torch.optim.Adam(
    list(model.parameters()) + list(v_proj.parameters()) + list(t_proj.parameters()),
    lr=initial_lr
)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=4000, gamma=0.5)

# 데이터로더 구성
dataset = FeatureDataset(image_feat, v_prime_feat, v_prime_feat, text_feat, t_prime_feat, class_id, real_or_fake)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [38]:
class LearnWModel(torch.nn.Module):
    def __init__(self, input_size=64, out_dim=64):
        super().__init__()
        self.W = torch.nn.Linear(input_size, out_dim, bias=False)

    def forward(self, *features):
        return [self.W(x) for x in features]

In [39]:
model = LearnWModel().to(device)
# v_proj, t_proj는 위와 동일
optimizer = torch.optim.Adam(list(model.parameters()) + list(v_proj.parameters()) + list(t_proj.parameters()), lr=1e-4)

In [ ]:
for epoch in tqdm(range(num_epochs), desc="Training Learn W"):
    for batch in dataloader:
        img_i, img_t, img_ti, txt_i, txt_t, *_ = [x.to(device) for x in batch]

        img_i = F.normalize(v_proj(img_i), dim=-1)
        img_t = F.normalize(v_proj(img_t), dim=-1)
        img_ti = F.normalize(v_proj(img_ti), dim=-1)
        txt_i = F.normalize(t_proj(txt_i), dim=-1)
        txt_t = F.normalize(t_proj(txt_t), dim=-1)

        vi, vt, vti, ti, tt = model(img_i, img_t, img_ti, txt_i, txt_t)

        L1 = symmetric_cross_entropy(vi, ti, device, 0.07)
        L2 = symmetric_cross_entropy(vt, tt, device, 0.07)
        L3 = symmetric_cross_entropy(vti, ti, device, 0.07)
        L4 = symmetric_cross_entropy(vti, tt, device, 0.07)
        L5 = symmetric_cross_entropy(vti, vt, device, 0.07)
        L6 = symmetric_cross_entropy(vti, vi, device, 0.07)

        loss = (-L1 - L2 - L6 + L3 + L4 + L5)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item():.4f}")
    print(f"Epoch {epoch+1}, L1={L1.item():.4f}, L2={L2.item():.4f}, L3={L3.item():.4f}, "
      f"L4={L4.item():.4f}, L5={L5.item():.4f}, L6={L6.item():.4f}, Loss={loss.item():.4f}")

Training Learn W:   0%|          | 0/100 [00:00<?, ?it/s]

Training Learn W:   1%|          | 1/100 [00:01<02:05,  1.27s/it]

Epoch 1/100, Loss: -0.0788
Epoch 1, L1=2.3100, L2=2.2985, L3=2.3031, L4=2.2985, L5=2.2328, L6=2.3046, Loss=-0.0788


Training Learn W:   2%|▏         | 2/100 [00:01<01:31,  1.07it/s]

Epoch 2/100, Loss: -0.1389
Epoch 2, L1=2.3413, L2=2.2867, L3=2.3152, L4=2.2867, L5=2.1885, L6=2.3013, Loss=-0.1389


...

Epoch 98/100, Loss: -0.0217
Epoch 98, L1=2.3379, L2=2.8304, L3=2.4220, L4=2.8304, L5=2.4790, L6=2.5848, Loss=-0.0217


Training Learn W:  99%|█████████▉| 99/100 [00:57<00:00,  1.59it/s]

Epoch 99/100, Loss: -159.0455
Epoch 99, L1=158.2836, L2=6.4189, L3=28.0838, L4=6.4189, L5=7.0080, L6=35.8538, Loss=-159.0455


Training Learn W: 100%|██████████| 100/100 [00:58<00:00,  1.71it/s]

Epoch 100/100, Loss: -3.0374
Epoch 100, L1=2.3169, L2=2.9169, L3=2.9184, L4=2.9169, L5=3.4153, L6=7.0542, Loss=-3.0374


In [41]:
torch.save(model.state_dict(), "./disentangle/learn_W.pth")

In [42]:
class ForgetWModel(torch.nn.Module):
    def __init__(self, input_size=64, out_dim=64):
        super().__init__()
        self.W = torch.nn.Linear(input_size, out_dim, bias=False)

    def forward(self, *features):
        return [self.W(x) for x in features]

In [43]:
model = ForgetWModel().to(device)
optimizer = torch.optim.Adam(list(model.parameters()) + list(v_proj.parameters()) + list(t_proj.parameters()), lr=1e-4)

In [ ]:
for epoch in tqdm(range(num_epochs), desc="Training Forget W"):
    for batch in dataloader:
        img_i, img_t, img_ti, txt_i, txt_t, *_ = [x.to(device) for x in batch]

        img_i = F.normalize(v_proj(img_i), dim=-1)
        img_t = F.normalize(v_proj(img_t), dim=-1)
        img_ti = F.normalize(v_proj(img_ti), dim=-1)
        txt_i = F.normalize(t_proj(txt_i), dim=-1)
        txt_t = F.normalize(t_proj(txt_t), dim=-1)

        vi, vt, vti, ti, tt = model(img_i, img_t, img_ti, txt_i, txt_t)

        L1 = symmetric_cross_entropy(vi, ti, device)
        L2 = symmetric_cross_entropy(vt, tt, device)
        L3 = symmetric_cross_entropy(vti, ti, device)
        L4 = symmetric_cross_entropy(vti, tt, device)
        L5 = symmetric_cross_entropy(vti, vt, device)
        L6 = symmetric_cross_entropy(vti, vi, device)

        loss = (L1 + L2 + L6 - L3 - L4 - L5)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item():.4f}")
    print(f"Epoch {epoch + 1}, L1={L1.item():.4f}, L2={L2.item():.4f}, L3={L3.item():.4f}, L4={L4.item():.4f}, L5={L5.item():.4f}, L6={L6.item():.4f}, Loss={loss.item():.4f}")

Training Forget W:   1%|          | 1/100 [00:00<00:47,  2.09it/s]

Epoch 1/100, Loss: -0.0055
Epoch 1, L1=2.2966, L2=2.3028, L3=2.3025, L4=2.3028, L5=2.3019, L6=2.3024, Loss=-0.0055


Training Forget W:   2%|▏         | 2/100 [00:00<00:46,  2.11it/s]

Epoch 2/100, Loss: -0.0726
Epoch 2, L1=2.2280, L2=2.3032, L3=2.3034, L4=2.3032, L5=2.3018, L6=2.3046, Loss=-0.0726


Training Forget W:   3%|▎         | 3/100 [00:01<00:41,  2.33it/s]

...

Epoch 98/100, Loss: -2015.8403
Epoch 98, L1=6.8997, L2=16.4761, L3=2026.1582, L4=16.4761, L5=7.0775, L6=10.4957, Loss=-2015.8403


Training Forget W:  99%|█████████▉| 99/100 [00:43<00:00,  2.39it/s]

Epoch 99/100, Loss: -1843.6803
Epoch 99, L1=7.2027, L2=1850.6130, L3=1852.4027, L4=1850.6130, L5=7.0041, L6=8.5239, Loss=-1843.6803


Training Forget W: 100%|██████████| 100/100 [00:43<00:00,  2.30it/s]

Epoch 100/100, Loss: -2017.2921
Epoch 100, L1=12.4930, L2=77.7202, L3=2035.1296, L4=77.7202, L5=3.1650, L6=8.5095, Loss=-2017.2921


In [45]:
torch.save(model.state_dict(), "./disentangle/forget_W.pth")